# Learning from complementary labels, for Cifar10 with 3 labels

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import Input, GlobalAveragePooling2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import EarlyStopping, CSVLogger
from keras import metrics

import os
import numpy as np
import math

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Experiment with three classes

In [2]:
num_classes = 3

### Prepare dataset

In [3]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# generate complementary labels for training targets
y_train3_pos = np.where(y_train<=2)[0]
x_train3 = x_train[y_train3_pos]
y_train3 = y_train[y_train3_pos]

y_ctrain3 = np.zeros(len(y_train3)).reshape(len(y_train3), 1)
for i,v in enumerate(y_train3):
    if v == 0:
        y_ctrain3[i] = np.random.choice([1,2],1)
    elif v == 1:
        y_ctrain3[i] = np.random.choice([0,2],1)
    elif v == 2:
        y_ctrain3[i] = np.random.choice([0,1],1)

In [6]:
# generate complementary labels for test targets
y_test3_pos = np.where(y_test<=2)[0]
x_test3 = x_test[y_test3_pos]
y_test3 = y_test[y_test3_pos]

y_ctest3 = np.zeros(len(y_test3)).reshape(len(y_test3), 1)
for i,v in enumerate(y_test3):
    if v == 0:
        y_ctest3[i] = np.random.choice([1,2],1)
    elif v == 1:
        y_ctest3[i] = np.random.choice([0,2],1)
    elif v == 2:
        y_ctest3[i] = np.random.choice([0,1],1)

In [7]:
print('x_train3 shape:', x_train3.shape)
print('y_ctrain3 shape:', y_ctrain3.shape)
print('x_test3 shape:', x_test3.shape)
print('y_ctest3 shape:', y_ctest3.shape)
print(x_train3.shape[0], 'train3 samples')
print(x_test3.shape[0], 'test3 samples')

x_train3 shape: (15000, 32, 32, 3)
y_ctrain3 shape: (15000, 1)
x_test3 shape: (3000, 32, 32, 3)
y_ctest3 shape: (3000, 1)
15000 train3 samples
3000 test3 samples


In [8]:
# prepare x dataset
x_train3 = x_train3.astype('float32')
x_test3 = x_test3.astype('float32')
x_train3 /= 255
x_test3 /= 255

# Convert ordinary class vectors to binary class matrices.
y_train3 = keras.utils.to_categorical(y_train3, num_classes)
y_test3 = keras.utils.to_categorical(y_test3, num_classes)

# Convert complementary class vectors to binary class matrices.
y_ctrain3 = keras.utils.to_categorical(y_ctrain3, num_classes)
y_ctest3 = keras.utils.to_categorical(y_ctest3, num_classes)

print('y_train3: {0}\ny_test3: {1}'.format(y_train3.shape, y_test3.shape))
print('y_ctrain3: {0}\ny_ctest3: {1}'.format(y_ctrain3.shape, y_ctest3.shape))

y_train3: (15000, 3)
y_test3: (3000, 3)
y_ctrain3: (15000, 3)
y_ctest3: (3000, 3)


## Convolutional neural network, in VGG

In [38]:
def vgg(x_train,classes):
    inputs = Input(shape=x_train.shape[1:])

    x = Conv2D(64, (3,3), padding="same")(inputs)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(128, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Conv2D(256, (3,3), padding="same")(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

### For ordinary classes

In [41]:
# load model for ordinary class
model = vgg(x_train3, num_classes)

batch_size = 32
epochs = 100

In [40]:
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar3_normal.csv", separator=',', append=True)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-5, amsgrad=True),
              metrics=['accuracy'])

In [42]:
# define image generator for data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train3)

In [43]:
# train
history = model.fit_generator(datagen.flow(x_train3, y_train3,
                                           batch_size=batch_size),
                              steps_per_epoch=x_train3.shape[0] // batch_size,
                              epochs=epochs,
                              validation_data=(x_test3, y_test3),
                              callbacks=[es_cb, csv_log])

Epoch 1/100
468/468 [==============================] - 31s 65ms/step - loss: 0.8914 - acc: 0.6824 - val_loss: 0.7338 - val_acc: 0.7597
Epoch 2/100
468/468 [==============================] - 23s 48ms/step - loss: 0.6291 - acc: 0.7611 - val_loss: 0.5227 - val_acc: 0.8097
Epoch 3/100
468/468 [==============================] - 23s 48ms/step - loss: 0.5331 - acc: 0.7943 - val_loss: 0.5015 - val_acc: 0.8130
Epoch 4/100
468/468 [==============================] - 22s 48ms/step - loss: 0.4743 - acc: 0.8134 - val_loss: 0.4058 - val_acc: 0.8460
Epoch 5/100
468/468 [==============================] - 23s 48ms/step - loss: 0.4405 - acc: 0.8277 - val_loss: 0.3887 - val_acc: 0.8633
Epoch 6/100
468/468 [==============================] - 23s 48ms/step - loss: 0.4020 - acc: 0.8475 - val_loss: 0.3919 - val_acc: 0.8623
Epoch 7/100
468/468 [==============================] - 23s 48ms/step - loss: 0.3753 - acc: 0.8549 - val_loss: 0.3843 - val_acc: 0.8670
Epoch 8/100
468/468 [==============================] - 

### Evaluate ordinary classification

In [45]:
# evaluate
evaluation = model.evaluate(x_test3, y_test3)
print("TEST Loss: {0}\tAccuracy: {1}".format(evaluation[0], evaluation[1]))

3000/3000 [==============================] - 1s 345us/step
TEST Loss: 0.2591986161569754	Accuracy: 0.9143333333333333


### For complementary classes

In [52]:
# load model for complementary label
cmodel = vgg(x_train3, num_classes)

batch_size = 32
epochs = 100

In [53]:
def sigmoid_loss(target, output):
    return 10 / (1 + math.e ** K.categorical_crossentropy(target, output))

In [54]:
def caccuracy(target, output):
    return 1 - metrics.categorical_accuracy(target, output)

In [55]:
es_cb = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
csv_log = CSVLogger("Cifar3_complementary.csv", separator=',', append=True)

cmodel.compile(loss=sigmoid_loss,
               optimizer=Adam(lr=0.0001, decay=1e-5, amsgrad=True),
               metrics=[caccuracy])

In [56]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    zca_whitening=False,
    rotation_range=15, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False) 

datagen.fit(x_train3)

In [57]:
# train
history = cmodel.fit_generator(datagen.flow(x_train3, y_ctrain3,
                                            batch_size=batch_size),
                               steps_per_epoch=x_train3.shape[0] // batch_size,
                               epochs=epochs,
                               validation_data=(x_test3, y_ctest3),
                               callbacks=[es_cb, csv_log])

Epoch 1/100
468/468 [==============================] - 32s 68ms/step - loss: 0.9263 - caccuracy: 0.8319 - val_loss: 0.7623 - val_caccuracy: 0.8620
Epoch 2/100
468/468 [==============================] - 23s 49ms/step - loss: 0.7758 - caccuracy: 0.8553 - val_loss: 0.7462 - val_caccuracy: 0.8607
Epoch 3/100
468/468 [==============================] - 23s 49ms/step - loss: 0.6941 - caccuracy: 0.8726 - val_loss: 0.6534 - val_caccuracy: 0.8823
Epoch 4/100
468/468 [==============================] - 23s 50ms/step - loss: 0.6350 - caccuracy: 0.8840 - val_loss: 0.5793 - val_caccuracy: 0.8940
Epoch 5/100
468/468 [==============================] - 24s 50ms/step - loss: 0.6021 - caccuracy: 0.8886 - val_loss: 0.5882 - val_caccuracy: 0.8920
Epoch 6/100
468/468 [==============================] - 23s 49ms/step - loss: 0.5726 - caccuracy: 0.8946 - val_loss: 0.5076 - val_caccuracy: 0.9080
Epoch 7/100
468/468 [==============================] - 23s 49ms/step - loss: 0.5531 - caccuracy: 0.8987 - val_loss: 0.

### evaluate complementary model

In [58]:
cpred = cmodel.predict(x_test3)

score = 0
for i in range(len(y_test3)):
    if np.argmax(y_test3[i]) == np.argmax(cpred[i]):
        score += 1
print(score / len(y_test3))

0.859
